# runtime.net_process

> TODO fill in description

In [ ]:
#| default_exp runtime.net_process

In [ ]:
#| hide
from nbdev.showdoc import *; 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
import asyncio
from abc import ABC, abstractmethod
from types import MappingProxyType
from typing import Type, Tuple, Dict

import fbdev
from fbdev.exceptions import NodeError, EdgeError
from fbdev.comp.packet import Packet
from fbdev.comp.port import PortType, PortSpec, PortSpecCollection, PortID, PortCollection
from fbdev.comp.base_component import BaseComponent
from fbdev.graph.graph_spec import GraphSpec, NodeSpec
from fbdev.graph.packet_registry import TrackedPacket
from fbdev.graph.net import Edge, Node, Net
from fbdev.graph.graph_component import GraphComponentFactory
from fbdev.runtime.base_net_runtime import BaseNetRuntime

In [ ]:
#|hide
from fbdev.complib import ExecComponent
from fbdev.complib.func_component_factory import func_component

In [ ]:
#|hide
show_doc(fbdev.runtime.net_process.NetProcess)

In [ ]:
#|export
class NetProcess(BaseNetRuntime):
    def __init__(self, net:Net, stop_port:PortID=None):
        super().__init__()
        self._net:Net = net
        self._stop_port = stop_port
        self._stop_listener_task = None
        
        if self._stop_port is not None and self._stop_port not in self._net.ports:
            raise ValueError(f"Port {self._stop_port} does not exist in net.")

    @property
    def ports(self) -> PortCollection: return self._net.ports

    def start(self):
        """Note: this method cannot be run from within an event loop."""
        super().start()
        raise NotImplementedError("NetProcess does not support synchronous execution.")
    
    async def astart(self):
        await super().astart()
        
        if self._stop_port is not None:
            async def stop_listener():
                try:
                    await self._net.ports[self._stop_port].get()
                    if not self._stopped: await self.stop()
                except asyncio.CancelledError: pass
            self._stop_listener_task = asyncio.create_task(stop_listener())
            
        await self._net.start()
        self._started = True
    
    async def await_stop(self):
        # Tried doing this by creating an event self._stop_event.
        # For some reason this would cause all tasks to just hang forever.
        # So instead, we just wait for the net to stop.
        # Really perplexing...
        await self._net.task_manager.exec_coros(self._net.states.stopped.wait(True), print_all_exceptions=False)
    
    async def stop(self):
        await super().stop()
        if self._stop_listener_task is not None:
            self._stop_listener_task.cancel()
            try: await self._stop_listener_task
            except asyncio.CancelledError: pass
        await self._net.task_manager.exec_coros(self._net.stop(), print_all_exceptions=False)
        self._stopped = True
        
    async def await_message(self, name:str):
        await self._net.task_manager.exec_coros(self._net.await_message(name), print_all_exceptions=False)

In [ ]:
class FooComponent(ExecComponent):
    port_specs = PortSpecCollection(
        PortSpec(PortType.MESSAGE, 'stop'),
    )
    async def _execute(self):
        print("Stopping")
        await self.send_message('stop')
        
async with NetProcess.from_component(FooComponent, stop_port=(PortType.MESSAGE, 'stop')) as ex:
    await ex.astart()
    await ex.await_stop()

In [ ]:
@func_component(loop_execution=True)
def SquareComponent(inp):
    return inp**2

async with NetProcess.from_component(SquareComponent) as ex:
    await ex.astart()
    
    await ex.ports.input.inp.put_value(2)
    print("2^2 =",await ex.ports.output.out.get_and_consume())
    
    await ex.ports.input.inp.put_value(5)
    print("5^2 =",await ex.ports.output.out.get_and_consume())
    
    await ex.ports.input.inp.put_value(7)
    print("7^2 =",await ex.ports.output.out.get_and_consume())

In [ ]:
@func_component(loop_execution=False)
def ExceptionComponent() -> None:
    raise Exception()

try:
    async with NetProcess.from_component(ExceptionComponent) as ex:
        await ex.aexecute()
        await ex.await_message('executed')
except Exception as e:
    print("Exception raised")